## What is Club Hub?

Club Hub is a comprehensive web application designed to centralize all club-related information for Del Norte High School. It streamlines the student experience by providing features such as club matching, updates on club events and announcements, and personalized profiles. With Club Hub, managing and exploring club activities has never been easier—bringing all your club-related endeavors together in one convenient platform.

## Individual Feature Overview

My feature is called club creator. It is a page on the website where users can create new clubs by inputting the club name, club description, and choosing club topics from a preset checklist of topics. When the user submits the form, the data will be stored in the backend database schema and the club will be created. Multiple clubs with the same name cannot be created. This feature is essentially the backbone of our project because many of our other features utilize the club creator feature such as joining/leaving clubs, leadership roles, and matching users to clubs based on their similar interests. Ultimately, the feature should work as follows:

1. Users open up the submission form and input data for a new club they would like to create. 
2. After submission, the club data should be posted into backend schema and displayed on the frontend as a new club in the club container.
3. For each club in the club box, they have their own card that displays the club information as well as the creator of the club and members and member count.
4. There is a delete button on the club card that allows only the creator of the club to delete it (authentication, @tokenrequired)
5. There is an update button on the club card that only allows the creator to update the club information
6. The delete and update buttons on the club cards only display for the clubs that the user created, otherwise the club cards won't have delete or update.

## Big Idea 2: Use of Lists, Dictionaries, and Database

**Lists (Rows)**: Lists are used to represent collections of objects, such as clubs retrieved from the database or topics selected in the front-end form.  

In [ ]:
clubs = Club.query.all()  # Retrieves all clubs from the database
return jsonify([club.to_dict() for club in clubs])  # Converts the list to JSON

**Dictionaries (Columns)**: Dictionaries are employed to structure data for communication between the API and the front-end.

In [ ]:
def to_dict(self):
    return {
        "id": self.id,
        "name": self.name,
        "description": self.description,
        "topics": self.topics,
        "user_id": self.user_id
    }

**Database**: SQLAlchemy is used to manage the database, translating Python objects to rows and columns in the database table.

In [ ]:
class Club(db.Model):
    __tablename__ = 'clubs'
    id = db.Column(db.Integer, primary_key=True)
    name = db.Column(db.String(255), unique=True, nullable=False)
    description = db.Column(db.String(255), nullable=False)
    topics = db.Column(db.JSON, nullable=False)
    user_id = db.Column(db.String(255), db.ForeignKey('users._uid'), nullable=False)


## Formatting Response Data (JSON) into DOM

**API Response to DOM Update**: JSON responses from the API are parsed on the front-end and rendered dynamically in the DOM.

In [ ]:
// Send a POST request to the backend
const response = await fetch('http://127.0.0.1:8887/api/club', {
    method: 'POST',
    headers: {
        'Content-Type': 'application/json',
        'Authorization': `Bearer ${localStorage.getItem('token')}` // will modify later with JWT token reqs
    },
    body: JSON.stringify(payload)
});

// more code to put data into club box was here as well as delete functionality

const response = await fetch('http://127.0.0.1:8887/api/club', { 
    method: 'GET', 
    headers: { 
        'Authorization': `Bearer ${localStorage.getItem('token')}` 
    } 
});
const clubs = await response.json();
clubs.forEach(club => {
    const clubBox = document.createElement('div');
    clubBox.classList.add('club-box');
    clubBox.innerHTML = `
        <h3>${club.name}</h3>
        <p><strong>Description:</strong> ${club.description}</p>
        <p><strong>Topics:</strong> ${club.topics.join(', ')}</p>
    `;
    clubListContainer.appendChild(clubBox);
});

**Parsing Topics (List)**: The topics field, stored as a JSON array in the database, is parsed and displayed as a comma-separated string.

In [ ]:
<p><strong>Topics:</strong> ${club.topics.join(', ')}</p>

**Error Handling**: Error messages returned in JSON format are displayed as alerts in the UI.

In [ ]:
if (!response.ok) {
    const error = await response.json();
    alert(`Error: ${error.message}`);
}

## Methods to Query and Modify Database (CRUD Operations)

**Extracting Rows (Query)**: Methods like Club.query.all() and Club.query.get(id) are used to fetch rows from the database as Python objects.

In [ ]:
clubs = Club.query.all()
return jsonify([club.to_dict() for club in clubs])

**Modifying Columns**: The create(), update(), and delete() methods in the Club class allow modification of individual columns in the database rows.

In [ ]:
# creates a new club in the database
def create(self):
    try:
        db.session.add(self)
        db.session.commit()
    except IntegrityError as e:
        db.session.rollback()
        logging.warning(f"IntegrityError: Could not create club '{self.name}' due to {str(e)}.")
        return None
    return self

# updates club data in database
def update(self, club_data):
    """
    Updates the club with new data.
    """
    try:
        for key, value in club_data.items():
            setattr(self, key, value)  # Dynamically update attributes of the club

        db.session.commit()
    except IntegrityError as e:
        db.session.rollback()
        logging.warning(f"IntegrityError: Could not update club '{self.name}' due to {str(e)}.")
        return None
    return self

# delete club from database
def delete(self):
    try:
        db.session.delete(self)
        db.session.commit()
    except IntegrityError:
        db.session.rollback()
        logging.warning(f"Could not delete club '{self.name}' due to IntegrityError.")
        return None

**Big Idea 1.4: Error Handling in CRUD**: Operations rollback in case of errors to maintain database integrity.

In [ ]:
if not data:
                return {'message': 'No input data provided'}, 400
            if 'name' not in data:
                return {'message': 'Club name is required'}, 400
            if 'description' not in data:
                return {'message': 'Club description is required'}, 400
            if 'topics' not in data:
                return {'message': 'Club topics are required'}, 400